Hi! In this notebook we will compare the linear regression to many regularization models as Ridge regression, Lasso, Elastic-Net, and others. If you not yet try a linear regression I advice you to look [this](https://www.kaggle.com/raisssaid/supervised-learning-linear-regression-advertising).

### Import libraries



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### Dataset

In [2]:
data = pd.read_csv(
    'https://raw.githubusercontent.com/saidRaiss/dataset/master/advertising_ads'
)

In [3]:
data.head()

,Unnamed: 0,TV,radio,newspaper,sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


In [4]:
data.drop(['Unnamed: 0'], axis=1, inplace=True)

In [6]:
X = data.drop(['sales', 'newspaper'], axis=1)
Y = data['sales'].values.reshape(-1,1)

In [7]:
X2 = sm.add_constant(X)
est = sm.OLS(Y, X2)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.897
Model:                            OLS   Adj. R-squared:                  0.896
Method:                 Least Squares   F-statistic:                     859.6
Date:                Tue, 27 Oct 2020   Prob (F-statistic):           4.83e-98
Time:                        19:24:42   Log-Likelihood:                -386.20
No. Observations:                 200   AIC:                             778.4
Df Residuals:                     197   BIC:                             788.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.9211      0.294      9.919      0.0

### Dataset splitting Train / Test

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.25, random_state=42
    )

### Useful functions

In [9]:
def print_evaluate(y_test, predicted):  
    mae = metrics.mean_absolute_error(y_test, predicted)
    mse = metrics.mean_squared_error(y_test, predicted)
    r2_square = metrics.r2_score(y_test, predicted)
    print('MAE:', mae)
    print('MSE:', mse)
    print('R2 Square', r2_square)
def cross_val(model, X = X, Y = Y, cv=10):
    MSE = cross_val_score(model, X, Y, scoring='neg_mean_squared_error', cv=cv)
    return print(-np.mean(MSE))

### Create models

In [10]:
# Linear Regression
linear_reg = LinearRegression()
# Ridge Regression
ridge = Ridge()
# Lasso
lasso = Lasso(tol=1e4)
# Elastic Net
elastic_net = ElasticNet()

### Linear Regression

In [11]:
linear_reg.fit(X_train, Y_train)
linear_reg_pred = linear_reg.predict(X_test)
print_evaluate(Y_test, linear_reg_pred)

MAE: 1.3886802126434374
MSE: 2.8539947557761023
R2 Square 0.8944787062601195


In [12]:
results = pd.DataFrame(
    data = [
            ["Linear Regression",
             metrics.mean_squared_error(Y_test, linear_reg_pred),
             metrics.r2_score(Y_test, linear_reg_pred)
             ]
            ],
            columns=['Model', 'Mean Squared Error', 'R2 Square'])

### Ridge Regression

In [13]:
parameters = {'alpha':[1e-15, 1e-10, 1e-8, 1e-4, 1e-3, 1e-2, 1, 5, 10, 20]}
ridgeCV = GridSearchCV(
    ridge, parameters, scoring='neg_mean_squared_error', cv=5
    )
ridgeCV.fit(X_train, Y_train)
ridgeCV_pred = ridgeCV.predict(X_test)
print_evaluate(Y_test, ridgeCV_pred)

MAE: 1.388680212643438
MSE: 2.853994755776102
R2 Square 0.8944787062601195


In [14]:
ridge_result = pd.DataFrame(
    data = [
            ["Ridge Regression",
             metrics.mean_squared_error(Y_test, ridgeCV_pred),
             metrics.r2_score(Y_test, ridgeCV_pred)
             ]
            ],
            columns=['Model', 'Mean Squared Error', 'R2 Square'])
results = results.append(ridge_result, ignore_index=True)

### Lasso

In [15]:
parameters =  {'alpha':[1e-15, 1e-10, 1e-8, 1e-4, 1e-3, 1e-2, 1, 5, 10, 20]}
lassoCV = GridSearchCV(
    lasso, parameters, scoring='neg_mean_squared_error', cv=5,
    )
lassoCV.fit(X_train, Y_train)
lassoCV_pred = lassoCV.predict(X_test)
print_evaluate(Y_test, lassoCV_pred)

MAE: 1.404140944428173
MSE: 2.846272038922038
R2 Square 0.8947642397468151


In [16]:
lasso_result = pd.DataFrame(
    data = [
            ["Lasso",
             metrics.mean_squared_error(Y_test, lassoCV_pred),
             metrics.r2_score(Y_test, lassoCV_pred)
             ]
            ],
            columns=['Model', 'Mean Squared Error', 'R2 Square'])
results = results.append(lasso_result, ignore_index=True)

### Elastic Net

In [17]:
elastic_net_default = ElasticNet()
elastic_net_default.fit(X_train, Y_train)
elastic_net_default_pred = elastic_net_default.predict(X_test)
print_evaluate(Y_test, elastic_net_default_pred)

MAE: 1.3836556179143498
MSE: 2.833966680898318
R2 Square 0.8952192081015983


In [18]:
elastic_net_default_result = pd.DataFrame(
    data = [
            ["Elastic Net (alpha=1)",
             metrics.mean_squared_error(Y_test, elastic_net_default_pred),
             metrics.r2_score(Y_test, elastic_net_default_pred)
             ]
            ],
            columns=['Model', 'Mean Squared Error', 'R2 Square'])
results = results.append(elastic_net_default_result, ignore_index=True)

In [19]:
elastic_net = ElasticNet(alpha = 10.5)
elastic_net.fit(X_train, Y_train)
elastic_net_pred = elastic_net.predict(X_test)
print_evaluate(Y_test, elastic_net_pred)

MAE: 1.3594171680347327
MSE: 2.80175986836401
R2 Square 0.8964099967388163


In [20]:
elastic_net_result = pd.DataFrame(
    data = [
            ["Elastic Net (alpha=10.5)",
             metrics.mean_squared_error(Y_test, elastic_net_pred),
             metrics.r2_score(Y_test, elastic_net_pred)
             ]
            ],
            columns=['Model', 'Mean Squared Error', 'R2 Square'])
results = results.append(elastic_net_result, ignore_index=True)

### Robust

In [21]:
from sklearn.linear_model import RANSACRegressor
from sklearn.preprocessing import PolynomialFeatures

In [22]:
ransac = RANSACRegressor()
ransac.fit(X_train, Y_train)
ransac_pred = ransac.predict(X_test)
print_evaluate(Y_test, ransac_pred)

MAE: 1.3720288245727554
MSE: 2.975007500550747
R2 Square 0.8900044789120171


In [23]:
ransac_result = pd.DataFrame(
    data = [
            ["Robust RANSAC Regressor",
             metrics.mean_squared_error(Y_test, ransac_pred),
             metrics.r2_score(Y_test, ransac_pred)
             ]
            ],
            columns=['Model', 'Mean Squared Error', 'R2 Square'])
results = results.append(ransac_result, ignore_index=True)

### Polynomial Features

In [24]:
poly_reg = PolynomialFeatures(degree=3)
X_poly = poly_reg.fit_transform(X)

X_poly_train, X_poly_test, y_poly_train, y_poly_test = train_test_split(
    X_poly, Y, test_size=0.25, random_state=42
    )

lin_reg_poly = LinearRegression()

lin_reg_poly.fit(X_poly_train,y_poly_train)
poly_pred = lin_reg_poly.predict(poly_reg.transform(X_test))
print_evaluate(Y_test, poly_pred)

MAE: 0.31296787989676117
MSE: 0.16079594134558928
R2 Square 0.994054860919219


In [25]:
poly_result = pd.DataFrame(
    data = [
            ["Linear Regression using Polynomial Features",
             metrics.mean_squared_error(Y_test, poly_pred),
             metrics.r2_score(Y_test, poly_pred)
             ]
            ],
            columns=['Model', 'Mean Squared Error', 'R2 Square'])
results = results.append(poly_result, ignore_index=True)

### Conclusion

In [26]:
results.sort_values(by=['Mean Squared Error', 'R2 Square'])

,Model,Mean Squared Error,R2 Square
6,Linear Regression using Polynomial Features,0.160796,0.994055
4,Elastic Net (alpha=10.5),2.801760,0.896410
3,Elastic Net (alpha=1),2.833967,0.895219
2,Lasso,2.846272,0.894764
1,Ridge Regression,2.853995,0.894479
0,Linear Regression,2.853995,0.894479
5,Robust RANSAC Regressor,2.975008,0.890004
